In [136]:
import cv2
from skimage import color
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
from scipy import ndimage as ni
import itertools
import pandas as pd
import math
from sklearn.cross_validation import cross_val_score
from sklearn import svm
import xgboost

In [137]:
def convolve(img):
    I_x = np.zeros((img.shape[0] - 2, img.shape[1] - 2))
    I_y = np.zeros((img.shape[0] - 2, img.shape[1] - 2))
    h = I_x.shape[0]
    w = I_y.shape[1]
    s_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s_y = np.array([[-1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    I_x = ni.convolve(img, s_x, mode='constant', cval=0.0)[1:-1, 1:-1]
    I_y = ni.convolve(img, s_y, mode='constant', cval=0.0)[1:-1, 1:-1]
    G = np.sqrt(I_x ** 2 + I_y ** 2)
    func = lambda x: x if x >= 0 else math.pi + x
    theta = (np.vectorize(func))(np.arctan2(I_y, I_x))
#     return , np.abs(np.arctan2(I_y, I_x))
    return G, theta

In [131]:
def histograms(img, cellRowsCnt=8, cellColsCnt=8, binCount=7, blockRowCells=5, blockColCells=5, step_x=3, step_y=3, eps=1e-10): 
    G , theta = convolve(img)
    #Итоговые фичи олжны быть одинакового размера
    '''
    (h, w) = theta.shape
    cellRows = np.floor(h / cellRowsCnt)
    cellCols = np.floor(w / cellColsCnt)
    img_hists = np.zeros((cellRowsCnt, cellColsCnt, binCount))
    for i in range(h - h % cellRowsCnt):
        for j in range(w - w % cellColsCnt):
            a = theta[i][j] * binCount / np.pi
            img_hists[i / cellRows, j / cellCols, a] += G[i][j]
    '''     
    hn = 8
    hm = 8
    cellRows = int(theta.shape[0] / hn)
    cellCols = int(theta.shape[1] / hm)
    H = np.zeros((hn, hm, binCount))
    bucket_angle = (math.pi + eps) / binCount
    buckets = np.floor(theta / bucket_angle)
    
    absGPerBucket = []
    for j in range(binCount):
        absGPerBucket.append(G * (buckets == j))
    
    for i in range(hn):
        for j in range(hm):
            for buck in range(binCount):
                H[i][j][buck] = np.sum(absGPerBucket[buck][i * cellRows : (i + 1) * cellRows, j * cellCols : (j + 1) * cellCols])
    return H
    

In [4]:
def descriptor(img_hists, blockRowCells=5, blockColCells=5, step_x=3, step_y=3, eps=1e-15):
    res = []
    for i in range(0, img_hists.shape[0], step_x):
        for j in range(0, img_hists.shape[1], step_y):
            v = np.concatenate(img_hists[i : i + blockRowCells, j : j + blockColCells, :])
            res.append(v / np.sqrt(np.sum(v ** 2) + eps))
    return np.concatenate(np.concatenate(res))

In [5]:
def extract_hog(img, cellRowsCnt=10, cellColsCnt=10, binCount=8, blockRowCells=4, blockColCells=4, step_x=2, step_y=2, eps=1e-10): 
    img_hists = histograms(img)
    return descriptor(img_hists)    

In [61]:
def prepare(img):
    (x, y) = img.shape
    i = x / 10
    j = y / 10
    return img[i:x-i, j:y-j]

In [138]:
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score

def fit_and_classify(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    clf = svm.LinearSVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(accuracy_score(y_test, y_pred))    

In [132]:
def f(x):
    res = str(x)
    return 'train/' + '0' * (5 - len(res)) + res + '.png'
filenames = [f(x) for x in range(39209)]

images = [color.rgb2gray(io.imread(x)) for x in filenames]

In [133]:
%%time
X = []
for j in tqdm(range(len(images))):
    X.append(extract_hog(prepare(images[j])))
y = pd.DataFrame(pd.read_csv('train/gt.csv')).class_id

100%|██████████| 39209/39209 [02:07<00:00, 307.19it/s]

CPU times: user 2min 7s, sys: 21.8 ms, total: 2min 7s
Wall time: 2min 7s


In [86]:
%%time
fit_and_classify(X, y)

0.869223707028
CPU times: user 32.9 s, sys: 75.8 ms, total: 33 s
Wall time: 32.9 s


### Попробуем обучить xgboost на подобранных параметрах
(Именно на нем параметры и подбирались, но вывод не сохранился, результат оказался хуже SVM)

In [140]:
Xdatatrain = xgboost.DMatrix(data = X_train, label = y_train)
Xdataval = xgboost.DMatrix(data = X_test, label = y_test)
Xdatatest = xgboost.DMatrix(X_test_test)
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')] 

In [ ]:
param = {}
param['booster'] = 'gbtree'
param['objective'] = 'multi:softmax'
param['eval_metric'] = 'merror'
param['silent'] = 0
param['num_class'] = 43
param['eta'] = 0.6

numround = 20

plst = list(param.items())

bst = xgboost.train(plst, Xdatatrain, numround, evals = watchlist,  early_stopping_rounds=50, verbose_eval = 1)
score = accuracy_score(y_test, bst.predict(Xdataval))
print("final score = {}".format(score))

### Тестовая выборка

In [147]:
def f(x):
    res = str(x)
    return 'test/' + '0' * (5 - len(res)) + res + '.png'
filenames_test = [f(x) for x in range(12342)]


images_test = [color.rgb2gray(io.imread(x)) for x in filenames_test]
X_test_test = []
for j in tqdm(range(len(images_test))):
    X_test_test.append(extract_hog(prepare(images_test[j])))

100%|██████████| 12342/12342 [00:39<00:00, 312.74it/s]


In [ ]:
%%time
clf = svm.LinearSVC()
clf.fit(X, y)
y_linear_pred = clf.predict(X_test_test)

In [150]:
def foo(x):
    res = str(x)
    return '0' * (5 - len(res)) + res + '.png'
test_files = [foo(x) for x in range(12342)]
y_df = pd.DataFrame(data={'filename': test_files, 'class_id': y_linear_pred})
y_df.to_csv('linear_res.csv', columns=['filename', 'class_id'], index=False)

In [143]:
y_xgb_pred = bst.predict(Xdatatest)

In [144]:
def foo(x):
    res = str(x)
    return '0' * (5 - len(res)) + res + '.png'
test_files = [foo(x) for x in range(12342)]
y_df = pd.DataFrame(data={'filename': test_files, 'class_id': y_xgb_pred})
y_df.to_csv('xgb_res.csv', columns=['filename', 'class_id'], index=False)